# Applied Process Mining Module

This notebook is part of an Applied Process Mining module. The collection of notebooks is a *living document* and subject to change. 

# Lecture 4 - 'Predictive Process Mining' (Python / PM4Py)

## Setup

<img src="https://pm4py.fit.fraunhofer.de/static/assets/images/pm4py-site-logo-padded.png" alt="PM4Py" style="width: 200px;"/>

In this notebook, we are using the [PM4Py library](https://pm4py.fit.fraunhofer.de/) in combination with several standard Python data science libraries:

* [pandas](https://pandas.pydata.org/)
* [PyTorch](https://pytorch.org/)

In [ ]:
## Perform the commented out commands to install the dependencies
# %pip install pandas
# %pip install matplotlib
# %pip install pm4py
# %pip install torch
# %pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
import pm4py
import os
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm

# Predictive Process Mining

## Event Log 

We are using the Sepsis event log as an example.

In [ ]:
from urllib.request import urlretrieve
import os

# download from 4tu.nl
urlretrieve('https://data.4tu.nl/ndownloader/files/24061976', 'sepsis.xes.gz')
sepsis_log = pm4py.read_xes('sepsis.xes.gz')
os.unlink('sepsis.xes.gz') # clean up

In [ ]:
len(sepsis_log)

## Prefix Extraction

Many different prediction tasks are possible based on an event log. Often, the assumption is made that only a prefix of a trace is known and that a prediction on some future state of the process instance represented by that trace should be made.

The first step is to generate suitable prefixes of the traces contained in the event log to be used as the training samples. As a *simple example*, we may be interested in predicting whether the patient in the process returns ot the emergency room indicated by the event *Return ER* as the last event. Since the event *Return ER* is part of the event log, we need to remove that event and remember in which trace it occurred. 

In [ ]:
sepsis_returns = [len(list(filter(lambda e: e["concept:name"] == "Return ER" ,trace))) > 0 for trace in sepsis_log]

In [ ]:
# check if this worked
print(sepsis_log[3][-1])
print(sepsis_returns[3])

print(sepsis_log[0][-1])
print(sepsis_returns[0])

At the same time, we may be interested in how well we can predict whether a patient returns for different sizes of the prefix, e.g., we can generate a new event log keeping only prefixes of each trace with at most size 10 (*10-prefix*).

**Note that this is just a simple example with 10 chosen as arbitrary prefix length and in the general case you generate not only prefixes of a specific size but of variables or all sizes. Also, some traces are less than 10 events long in which case we would use the full trace for the prediction, which would not be very useful in practice.**

In [ ]:
# remove Return ER event
sepsis_log = pm4py.filter_event_attribute_values(sepsis_log, "concept:name", "Return ER", level = "event", retain=False)

from pm4py.objects.log.obj import EventLog, Trace
# generate prefixes, note that we need to add the casts to EventLog and Trace to make sure that the result is a PM4Py EventLog object
sepsis_prefixes = EventLog([Trace(trace[0:10], attributes = trace.attributes) for trace in sepsis_log])

In [ ]:
# check the trace length
print([len(trace) for trace in sepsis_log][0:15])
print([len(trace) for trace in sepsis_prefixes][0:15])

## Prefix Encoding

For training a prediction model, the traces or sequences of events need to be often transformed to a vector representation. We show how to compute three basic encodings+ using the built-in PM4Py [feature selection and processing](https://pm4py.fit.fraunhofer.de/documentation#decision-trees) functionality.

Of course, more complex encodings such as representing each trace as a sequence of features are possible, e.g., for sequential models such as LSTMs. This is left as exercise. 

### Feature Selection \& Engineering

Before we do prefix encoding, we need to select which features we will use for the prediction. In this example we will only use the "activity" of the events as feature. Depending on your prediction problem, you might want to include additional trace/event attributes.

Additionally, you can also derive new trace-level features (e.g., day of week, time since case start) or log-based features (e.g., workload of resources, number of active cases at a certain time). This is left as exercise.

### Encoding as Set of Events

In [ ]:
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features

# log_to_feature provides a flexible interface to compute features on an event and trace level
# see the documentation for more information: https://pm4py.fit.fraunhofer.de/documentation#item-7-0-2 
data, feature_names = log_to_features.apply(sepsis_prefixes, parameters={"str_ev_attr": ["concept:name"]})

The standard encoding of the `concept:name` attribute (i.e., the event label) is a one-hot encoded vector. Let us have a look at the encoding. The index of the number corresponds to the index in the feature label vector.

In [ ]:
from pm4py.objects.log.util.log import project_traces
def project_nth(log, index):
    print(str(project_traces(log)[index]))

In [ ]:
project_nth(sepsis_prefixes, 0)

In [ ]:
print(feature_names)

In [ ]:
print(data[0])

The overall data shape is:

In [ ]:
np.asarray(data).shape

So, PM4Py gives us a *one-hot encoding* of the so called *set abstraction* of the event log. This means there are 16 distinct activities in the event log and the feature vector simply encodes whether that activity is present or not in the data. 

Let us have a look at the distribution of these feature vectors:

In [ ]:
# look at the unique vectors and their occurrence frequency/count
dist_features = np.unique(data, return_counts= True, axis = 0)
print(dist_features)

What is the most common feature vector?

In [ ]:
# argmax give use the index of the most frequent vector
dist_features[0][np.argmax(dist_features[1])]

Makes sense, almost all activities actually are bound to occur in this process. There are only few choices.
So, this encoding is likely not the most useful one but a very simple one.

### Encoding as Bi-Grams / Succession Relation

In [ ]:
data_2gram, feature_names = log_to_features.apply(sepsis_prefixes, 
                                                  parameters={"str_ev_attr": [], 
                                                        "str_tr_attr": [], 
                                                        "num_ev_attr": [], 
                                                        "num_tr_attr": [], 
                                                        "str_evsucc_attr": ["concept:name"]})
feature_names

Each feature represents the succession relation (or bigram) between any two activities of the event log. We transform the features into a tensor.

In [ ]:
data_2gram = np.asarray(data_2gram)

Let us, again, have a look at the encoding of the first trace.

In [ ]:
project_nth(sepsis_log, 0)

In [ ]:
print(data_2gram[0])

### Encoding as Bag of Words / Multiset of Events

Another option would be to use the encoding known as [bag-of-words model](https://en.wikipedia.org/wiki/Bag-of-words_model) in Natural Language Processing, which is constructing a multiset of the one-hot encoded events. So, the frequency with which each activity occurs is reflected. This encoding is not provided in PM4Py but can be easily computed with Pandas and Numpy.

We first need to transform the PM4Py event log to a Pandas data frame.

In [ ]:
sepsis_df = pm4py.convert_to_dataframe(sepsis_prefixes)
sepsis_df.head(25)

We build a bag of words representation by grouping our data and then counting the number of events refering to the individual activities.

In [ ]:
# concept:name refers to the activity
# case:concept:name refers to the case identifier
sepsis_case_act = sepsis_df.loc[:,["case:concept:name", "concept:name"]]
sepsis_case_act

In [ ]:
# Count the occurrence of activities in a trace (no sorting to keep order of traces stable!)
sepsis_act_count = sepsis_case_act.groupby(["case:concept:name", "concept:name"], sort=False).size()
sepsis_act_count

We have the count of each activity for each trace and still need to convert this to a tensor format such that we have one feature vector (columns) per case (row).

In [ ]:
sepsis_bag = np.asarray(sepsis_act_count.unstack(fill_value=0))
sepsis_bag

In [ ]:
sepsis_bag.shape

Let us, again, have a look at the encoding of the first trace.

In [ ]:
project_nth(sepsis_log, 0)
print(sepsis_bag[0])

In [ ]:
project_nth(sepsis_log, 1)
print(sepsis_bag[1])

This already gives us much more information to work with.

## Prediction

Let us try to build a basic prediction model based on this information. In this example, we aim to predict the binary outcome whether the event `Return ER` occurred or not. 

**Disclaimer: here *basic* means that the model and encoding is not expected to be of any quality. Also note that the prediction task, while useful, may not be feasible based on the prefix encoding that we chose. Treat the following code as an example and starting point only!**

### Data Preparation

#### Target Variable

Let us look at the distribution of the target variable.

In [ ]:
np.unique(sepsis_returns, return_counts=True)

In [ ]:
# For future processing we need 0 and 1 instead of True and False
sepsis_returns = np.asarray(sepsis_returns).astype(int)
sepsis_returns.shape

#### Data Scaling & Loading

This often helps prediction models to perform better.

**Important:** make sure to not compute the scaling with the test set included since there is a risk of data leakage otherwise. In other words, the test set should be separated before any pre-processing, which may use a property of the dataset, is applied. Of course, the test set is scaled as well but with the scaler *trained* only on the training set. 

In [ ]:
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler

scaler_x = MinMaxScaler()
data_scaled = scaler_x.fit_transform(sepsis_bag)

scaler_y = FunctionTransformer() # for binary values scaling does not make sense at all but we keep it for symetry and apply the "NoOp" scaler
target_scaled = scaler_y.fit_transform(sepsis_returns.reshape(-1, 1))

### Model Definition

Let's define a simple network and try to overfit. We make use of PyTorch to build a simple Neural Network. 

**Disclaimer: Again, this is just a simple example and not in anyway meant as a recommendation for a model.**

In [ ]:
class NeuralNetworkBinaryOutcome(nn.Module):
    def __init__(self):
        super(NeuralNetworkBinaryOutcome, self).__init__()
        self.linear_relu_stack = nn.Sequential(            
            torch.nn.Linear(x.shape[1], 64),
            nn.BatchNorm1d(num_features=64),
            nn.LeakyReLU(),            
            torch.nn.Linear(64, 128),
            nn.BatchNorm1d(num_features=128),          
            torch.nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

We use a standard training loop in PyTorch:

In [ ]:
def train(dataloader, model, 
          loss_fn, measure_fn, 
          optimizer, device, epochs): 
    
    losses = []
    size = len(dataloader.dataset)
    
    loop = tqdm(range(epochs))
    
    for epoch in loop:
    
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()

            # Compute prediction error
            pred = model(X)
            
            loss = loss_fn(pred, y)
            measure = measure_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            
            losses.append([loss.item(), measure.item()])
            
        loop.set_description('Epoch {}/{}'.format(epoch + 1, epochs))
        loop.set_postfix(loss=loss.item(), measure=measure.item())
    
    return losses

And can use the following function to get all evaluation results:

In [ ]:
def evaluate_all(dataloader, model, device):    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    
    result = []
    original = []

    with torch.no_grad(): 
        for X, y in tqdm(dataloader):  
            X, y = X.to(device), y.to(device) 
            pred = model(X)          
                        
            result.extend(pred.flatten().numpy())
            original.extend(y.flatten().numpy())
                           
    return np.asarray(result), np.asarray(original)

### Training

Prepare the data for the PyTorch data loading mechanism.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# We need float32 data
x = torch.from_numpy(data_scaled.astype('float32'))
y = torch.from_numpy(target_scaled.astype('float32'))

# Always check the shapes
print(x.shape)
print(y.shape)

ds = TensorDataset(x, y)
train_dataloader = DataLoader(ds, batch_size=64, shuffle=True)

Let us check a random single sample from our data loader (always a good idea!) 

In [ ]:
inputs, classes = next(iter(train_dataloader))
print(inputs[0])
print(classes[0])

We train the model using cross entropy as loss function accuracy as easier to interpret measure to report.

In [ ]:
## if you want ot use a GPU you need to tweak the requirements.txt to include the GPU-enabled PyTorch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# fix a seed to get reproducible results
torch.manual_seed(42)

model = NeuralNetworkBinaryOutcome().to(device)
print(model)

def get_accuracy(y_prob, y_true):    
    y_true = y_true.flatten()
    y_prob = y_prob.flatten()
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return (y_true == y_prob).sum() / y_true.size(0)
measure_fn = get_accuracy

results = train(train_dataloader, model, 
                nn.BCELoss(), # crossentropy for binary target 
                get_accuracy, 
                torch.optim.Adam(model.parameters()), 
                device, 100)

In [ ]:
%matplotlib inline

results_data = pd.DataFrame(results)
results_data.columns = ['loss', 'measure']
ax = results_data.plot(subplots=True);

In [ ]:
print("Accuracy: " + str(results[len(results)-1][1]))

true_returns = np.unique(sepsis_returns, return_counts=True)[1][0]
true_not_returns = np.unique(sepsis_returns, return_counts=True)[1][1]

print("Accuracy (never returns)" + str(true_returns / len(sepsis_returns)))
print("Accuracy (always returns)" + str(true_not_returns / len(sepsis_returns)))

## Brief Evaluation

Ok, that is a bit better compared to simply always saying that the patient does not return. But the accuracy on the training set (**not even considering a test set!**) is still varying a lot and the variation of the log and accuracy over the epochs trained does not look good either. So, let us still have a look at the individual predictions and their score depending on the ground truth.

In [ ]:
test_dataloader = DataLoader(ds, batch_size=256, shuffle=False)
result, original = evaluate_all(test_dataloader, model, device)

In [ ]:
pd_pos = pd.DataFrame({'Returns': result[original == 1]})
pd_neg = pd.DataFrame({'Does not return': result[original == 0]})
pd.concat([pd_pos, pd_neg],axis=1).boxplot().set_ylabel('Score')

There seems to be some separation but likely the prediction model will give us many false positives when used to identify returning patients in practice.

**Of course, you should now compute the usual measures for classification tasks and the threshold for making a decision: recall, precision, confusion matrices, area under the curve and many other ways to deeply evaluate a prediction model. Always consider what would be the use case of your prediction.**

Why is this so bad? Let us have a look at the data distribution we put in:

In [ ]:
# count the unique vectors
dist_bags = np.unique(sepsis_bag, return_counts=True, axis=0)

# sort them with numpy
unique_vectors = dist_bags[0][np.argsort(-dist_bags[1])]
count_vectors = dist_bags[1][np.argsort(-dist_bags[1])]

pd.DataFrame({'Occurrence of unique sample vectors': count_vectors}).boxplot().set_ylabel('Frequency')

Many of the traces result in the exact same sample. Let us check what is the "return status" for the most common sample that represents more than 175 traces.

In [ ]:
# most frequently used vector
unique_vectors[0]

In [ ]:
# find the sample indicies for this vector
sample_indicies = np.where((sepsis_bag == unique_vectors[0]).all(axis=1)) 
sample_durations = target_scaled[sample_indicies]

In [ ]:
np.unique(sample_durations, return_counts=True)

It is clear that, without additional information, there is nothing the prediction model can learn to represent this division for the exact same feature values. We can look at further examples, but it seems we simply cannot reliably predict whether a patient will return from the bag-of-words / multiset of events model in the Sepsis event log.

This was just an example on how to use a predictive model with an event log to predict a binary process characteristic based on events contained in the event log.